In [50]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
from tqdm import tqdm
import os
import json

### Feature Extraction from Trulia

In [54]:
BASE_HEADERS = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
}

url_iter = ["https://www.trulia.com/p/ca/san-francisco/1531-funston-ave-san-francisco-ca-94122--2083026624" ,"https://www.trulia.com/p/ca/san-francisco/170-highland-ave-san-francisco-ca-94110--2082898130"] 

In [55]:
#Initialising List of dictionaries - each dictionary contains information about a house
house_dict_list = []
#Looping on each URL
for url_1 in url_iter:  
    #Could replace with opening html files
    response = requests.get(url_1, headers=BASE_HEADERS)
    soup_each_house = BeautifulSoup(response.content, 'html.parser')
    
    house_details_dict = {}
    house_details_dict['URL'] = url_1
    
    #Feature Extraction - higher level details in the header
    try:
        house_details_dict['media_count'] = soup_each_house.find('span', attrs={'data-testid' : "media-count"}).text
    except:
        house_details_dict['media_count'] = "NA"
    try:
        house_details_dict['Price'] = soup_each_house.find('div', attrs={'data-testid' : "home-price-details-medium"}).find('h3').text
    except:
        house_details_dict['Price'] = "NA"
    try:
        add_temp = soup_each_house.find('div', attrs={'data-testid':"home-details-summary-address"}).findAll('span')
        house_details_dict['Address'] = add_temp[0].text + ", " +add_temp[1].text
    except:
        house_details_dict['Address'] = "NA"
    try:
        house_details_dict['Last_Sold'] = soup_each_house.find('div', attrs={'class':"Text__TextBase-sc-1cait9d-0-div Text__TextContainerBase-sc-1cait9d-1 csrRqu gUnlde"}).text
    except:
        house_details_dict['Last_Sold'] = "NA"
    try:
        house_details_dict['Beds'] = soup_each_house.find('div', attrs={'data-testid':"home-summary-size-bedrooms"}).text
    except:
        house_details_dict['Beds'] = "NA"
    try:
        house_details_dict['Baths'] = soup_each_house.find('div', attrs={'data-testid':"home-summary-size-bathrooms"}).text
    except:
        house_details_dict['Baths'] = "NA"
    try:
        house_details_dict['Floor_Space'] = soup_each_house.find('div', attrs={'data-testid':"home-summary-size-floorspace"}).text
    except:
        house_details_dict['Floor_Space'] = "NA"
    
    #Updating dictionaries with information from the sections like interior details
    try:        
        table_list = soup_each_house.findAll('table', attrs={'class':'Table-latbb5-3 hMYIhS'})
        for table in table_list:
            heading = table.find('h3').text
            house_details_dict[heading] = {}
            for row in table.find('tbody').findAll('tr'):
                subheading = row.find('div').text
                row_text = []
                for i in row.findAll('span', attrs = {'class':"Feature__FeatureListItem-sc-w1mxt5-0 gmLKqq"}):
                    if ':' in i.text:
                        dict_temp = {}
                        dict_temp[i.text.split(':')[0].strip()] = i.text.split(':')[1].strip()
                        row_text.append(dict_temp)
                    else:
                        row_text.append(i.text)
                house_details_dict[heading][subheading] = row_text
            if heading == "Miscellaneous":
                break
    except Exception as e:
        print(e)
    house_dict_list.append(house_details_dict)
    time.sleep(10)

In [57]:
with open('test.json', 'w') as fout:
    json.dump(house_dict_list , fout)